In [20]:
import os
import sys
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch
import matplotlib.pyplot as plt
from Naive_DAE import Naive_DAE,Dropout_DAE
import AE_Stats
from load_data_fn import load_data,load_data_no_filter

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, TensorDataset
import importlib
import time
import ae_train
from losses import *
from telescope_torch import *

path = 'MIT_TTbar'
prefixed = [filename for filename in os.listdir(path) if filename.startswith("dt_norm")]

data = []
for p in prefixed:
    data.append([torch.load(f'{path}/{p}'),p])

In [2]:
path = '/uscms/home/nswood/nobackup/Notebooks/AE_Dev/models/batched_models/wafer_layer_split_mip_std_1_mean_nonzero'

models = os.listdir(path)[9:]

In [3]:

#all past 29
cur_train = []
for m in models:
    if int(m[8]) == 1: #and int(m[10:]) > 29:
        cur_train.append(m)

In [4]:
#Calculating MSE of trained models
path = '/uscms/home/nswood/nobackup/Notebooks/AE_Dev/models/batched_models/wafer_layer_split_mip_std_1_mean_nonzero'

cur_model = models[1]

def get_data(data,cur_model):
    cur_data = []
    for d in data:
        if d[1] == cur_model:
            cur_data = d[0]
            break
    return cur_data

test_set = []
train_set = []
pred_set = []

for m in cur_train:
    cur_model = torch.load(os.path.join(path,m)).to('cpu')
    cur_data = get_data(data,m).to('cpu')
    if len(cur_data)< 10000:
        train_set.append(cur_data[0:int(0.8*len(cur_data))])
        test_data = cur_data[-int(0.19*len(cur_data)):]
        test_set.append(test_data)
    else:
        train_set.append(cur_data[0:int(0.8*10000)])

        test_data = cur_data[-int(0.19*10000):]
        test_set.append(test_data)
    
    
    
    

In [5]:
device = 'cuda'
train_dt = torch.vstack(train_set).to(device)
test_dt = torch.vstack(test_set).to(device)

In [6]:
train_dt = train_dt[torch.count_nonzero(train_dt[:,0:48], dim=1) > 7]
test_dt = test_dt[torch.count_nonzero(test_dt[:,0:48], dim=1) > 7]

In [7]:
def loss_p(data,pred,epoch,mean,std,alpha = 0.2,cut = 100):
    s = torch.sum(data[:,0:48],dim=1)
    s[s ==0.0000] = 1
    return new_loss(data,pred,epoch,mean,std) + 0.25*np.abs(torch.mean(torch.sum(pred-data[:,0:48],dim=1)/s).item()*100)

In [8]:
mean = 0
std = 1

In [9]:
size_1_train = np.min([100000,len(train_dt)])
size_1_test = np.min([10000,len(test_dt)])
train_loc = train_dt
test_loc = test_dt
# train = torch.vstack([train,largest[0:1000]] )
train_loc=train_loc[torch.randperm(train_loc.size()[0])]
test_loc=test_loc[torch.randperm(test_loc.size()[0])]
train_loc_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(train_loc)),
    batch_size=100,
    shuffle=False
)

test_loc_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(test_loc)),
    batch_size=100,
    shuffle=False
)

train_1 = train_dt[:,0:48]
test_1 = test_dt[:,0:48]
# train = torch.vstack([train,largest[0:1000]] )
train_1=train_1[torch.randperm(train_1.size()[0])]
test_1=test_1[torch.randperm(test_1.size()[0])]
train_1_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(train_1)),
    batch_size=100,
    shuffle=False
)

test_1_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(test_1)),
    batch_size=100,
    shuffle=False
)

In [10]:
loss = new_loss

In [14]:
#fine-tune autoencoder
#batch 500

# model_1 = torch.load('models/dt_1_greater_0_450_250_100_dif_2').to(device)
# model_1.decoders.add_module('ReLU',nn.ReLU())
                        
# model_1 = Naive_DAE([48,250,150,16]).to(device)

# model_1 = torch.load('models/dt_1_high_occupancy_greater_7').to(device)

test = test_dt
lr = 3.5e-6
num_epochs = 500

optimizer = optim.Adam(model_1.parameters(), lr,weight_decay=5e-2)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = 0.5)

all_1_test_losses = []
all_1_train_losses = []
# train
running_loss = float("inf")
for epoch in range(num_epochs):
    break
    losses = []
    
    for i, data_list in enumerate(train_1_d1_flat):
        model_1.train()
        d = data_list[0]
        v_pred = model_1(d)
        
#         batch_loss = torch.mean(telescopeMSE2(data[:,0:48].to(device), v_pred.to(device)))
        batch_loss = new_loss(d[:,0:48].to(device), v_pred.to(device),epoch,mean, std, cut = 1)
        all_1_train_losses.append(batch_loss.item())
        losses.append(batch_loss.item())
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        
    scheduler.step(batch_loss)
    data_1_test = test[:,0:48]
    model_1.eval()
    test_pred = model_1(data_1_test)
    batch_1_test = new_loss(data_1_test, test_pred,epoch, mean,std, cut =1)
#     batch_1_test = torch.mean(telescopeMSE2(data_1_test[:,0:48].to(device), test_pred.to(device)))
    all_1_test_losses.append(batch_1_test.item())
    running_loss = np.mean(losses)
    running_1est_loss = batch_1_test.item()
    
    print('Epoch {}, lr {}'.format(
        epoch, optimizer.param_groups[0]['lr']))
    
    print(f"Epoch {epoch}: Train {running_loss}, Test {running_1est_loss}")
model_1.eval()
torch.save(model_1,'models/dt_1_high_occupancy_greater_7')

# AE_Stats.gen_all_stats(model_1(dt_1[-size_1_test:,0:48]).cpu(),dt_1[-size_1_test:,0:48].cpu(),dt_1[-size_1_test:].cpu())

In [15]:

#all past 29
cur_train = []
for m in models:
    if int(m[8]) == 2: #and int(m[10:]) > 29:
        cur_train.append(m)
test_set = []
train_set = []
pred_set = []

for m in cur_train:
    cur_model = torch.load(os.path.join(path,m)).to('cpu')
    cur_data = get_data(data,m).to('cpu')
    if len(cur_data)< 10000:
        train_set.append(cur_data[0:int(0.8*len(cur_data))])
        test_data = cur_data[-int(0.19*len(cur_data)):]
        test_set.append(test_data)
    else:
        train_set.append(cur_data[0:int(0.8*10000)])

        test_data = cur_data[-int(0.19*10000):]
        test_set.append(test_data)
train_dt = torch.vstack(train_set).to(device)
test_dt = torch.vstack(test_set).to(device)

train_dt = train_dt[torch.count_nonzero(train_dt[:,0:48], dim=1) > 7]
test_dt = test_dt[torch.count_nonzero(test_dt[:,0:48], dim=1) > 7]
size_1_train = np.min([100000,len(train_dt)])
size_1_test = np.min([10000,len(test_dt)])
train_loc = train_dt
test_loc = test_dt
# train = torch.vstack([train,largest[0:1000]] )
train_loc=train_loc[torch.randperm(train_loc.size()[0])]
test_loc=test_loc[torch.randperm(test_loc.size()[0])]
train_loc_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(train_loc)),
    batch_size=100,
    shuffle=False
)

test_loc_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(test_loc)),
    batch_size=100,
    shuffle=False
)

train_1 = train_dt[:,0:48]
test_1 = test_dt[:,0:48]
# train = torch.vstack([train,largest[0:1000]] )
train_1=train_1[torch.randperm(train_1.size()[0])]
test_1=test_1[torch.randperm(test_1.size()[0])]
train_1_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(train_1)),
    batch_size=100,
    shuffle=False
)

test_1_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(test_1)),
    batch_size=100,
    shuffle=False
)

#fine-tune autoencoder
#batch 500

model_1 = torch.load('models/dt_2_greater_0_450_250_100_dif_2').to(device)
model_1.decoders.add_module('ReLU',nn.ReLU())
                        
# model_1 = Naive_DAE([48,250,150,16]).to(device)

# model_1 = torch.load('models/dt_1_high_occupancy_greater_7').to(device)

test = test_dt
lr = 3.5e-6
num_epochs = 500

optimizer = optim.Adam(model_1.parameters(), lr,weight_decay=5e-2)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = 0.5)

all_1_test_losses = []
all_1_train_losses = []
# train
running_loss = float("inf")
for epoch in range(num_epochs):
    
    losses = []
    
    for i, data_list in enumerate(train_1_d1_flat):
        model_1.train()
        d = data_list[0]
        v_pred = model_1(d)
        
#         batch_loss = torch.mean(telescopeMSE2(data[:,0:48].to(device), v_pred.to(device)))
        batch_loss = new_loss(d[:,0:48].to(device), v_pred.to(device),epoch,mean, std, cut = 1)
        all_1_train_losses.append(batch_loss.item())
        losses.append(batch_loss.item())
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        
    scheduler.step(batch_loss)
    data_1_test = test[:,0:48]
    model_1.eval()
    test_pred = model_1(data_1_test)
    batch_1_test = new_loss(data_1_test, test_pred,epoch, mean,std, cut =1)
#     batch_1_test = torch.mean(telescopeMSE2(data_1_test[:,0:48].to(device), test_pred.to(device)))
    all_1_test_losses.append(batch_1_test.item())
    running_loss = np.mean(losses)
    running_1est_loss = batch_1_test.item()
    
    print('Epoch {}, lr {}'.format(
        epoch, optimizer.param_groups[0]['lr']))
    
    print(f"Epoch {epoch}: Train {running_loss}, Test {running_1est_loss}")
model_1.eval()
torch.save(model_1,'models/dt_2_high_occupancy_greater_7')

# AE_Stats.gen_all_stats(model_1(dt_1[-size_1_test:,0:48]).cpu(),dt_1[-size_1_test:,0:48].cpu(),dt_1[-size_1_test:].cpu())

Epoch 0, lr 3.5e-06
Epoch 0: Train 9.267121169721028, Test 7.994597434997559
Epoch 1, lr 3.5e-06
Epoch 1: Train 7.680721792936011, Test 7.046117782592773
Epoch 2, lr 3.5e-06
Epoch 2: Train 7.1594536384574985, Test 10.790297508239746
Epoch 3, lr 3.5e-06
Epoch 3: Train 6.595445045526477, Test 10.028594970703125
Epoch 4, lr 3.5e-06
Epoch 4: Train 6.192465578771548, Test 9.522397994995117
Epoch 5, lr 3.5e-06
Epoch 5: Train 5.881968006314968, Test 9.128978729248047
Epoch 6, lr 3.5e-06
Epoch 6: Train 5.631263243036936, Test 8.808578491210938
Epoch 7, lr 3.5e-06
Epoch 7: Train 5.422984712208684, Test 8.531396865844727
Epoch 8, lr 3.5e-06
Epoch 8: Train 5.236636594350159, Test 8.26663589477539
Epoch 9, lr 3.5e-06
Epoch 9: Train 5.06785728746095, Test 8.046697616577148
Epoch 10, lr 3.5e-06
Epoch 10: Train 4.9293934812030615, Test 7.851413249969482
Epoch 11, lr 3.5e-06
Epoch 11: Train 4.808588495681723, Test 7.669789791107178
Epoch 12, lr 3.5e-06
Epoch 12: Train 4.696949168941563, Test 7.4971103

Epoch 104, lr 3.5e-06
Epoch 104: Train 2.722377707992775, Test 3.995054006576538
Epoch 105, lr 3.5e-06
Epoch 105: Train 2.7171422291021887, Test 3.9850783348083496
Epoch 106, lr 3.5e-06
Epoch 106: Train 2.7120634924438787, Test 3.9748754501342773
Epoch 107, lr 3.5e-06
Epoch 107: Train 2.7068525887289536, Test 3.9650940895080566
Epoch 108, lr 3.5e-06
Epoch 108: Train 2.7019814465671033, Test 3.954298496246338
Epoch 109, lr 3.5e-06
Epoch 109: Train 2.696917085779514, Test 3.944687843322754
Epoch 110, lr 3.5e-06
Epoch 110: Train 2.6921780439233594, Test 3.935955286026001
Epoch 111, lr 3.5e-06
Epoch 111: Train 2.6873348506071824, Test 3.9261984825134277
Epoch 112, lr 3.5e-06
Epoch 112: Train 2.6830744434722327, Test 3.915146827697754
Epoch 113, lr 3.5e-06
Epoch 113: Train 2.6777795277565364, Test 3.90834641456604
Epoch 114, lr 3.5e-06
Epoch 114: Train 2.673140940223287, Test 3.8983561992645264
Epoch 115, lr 3.5e-06
Epoch 115: Train 2.6686468163654897, Test 3.8906922340393066
Epoch 116, lr 

Epoch 204, lr 3.5e-06
Epoch 204: Train 2.4065190874069575, Test 3.368114471435547
Epoch 205, lr 3.5e-06
Epoch 205: Train 2.4045399377154424, Test 3.3637094497680664
Epoch 206, lr 3.5e-06
Epoch 206: Train 2.402470082162397, Test 3.359891891479492
Epoch 207, lr 3.5e-06
Epoch 207: Train 2.400470223901143, Test 3.3561253547668457
Epoch 208, lr 3.5e-06
Epoch 208: Train 2.3985512023386746, Test 3.3519163131713867
Epoch 209, lr 3.5e-06
Epoch 209: Train 2.3966444986768076, Test 3.3475375175476074
Epoch 210, lr 3.5e-06
Epoch 210: Train 2.3947024550362537, Test 3.3441758155822754
Epoch 211, lr 3.5e-06
Epoch 211: Train 2.392786042727972, Test 3.3396897315979004
Epoch 212, lr 3.5e-06
Epoch 212: Train 2.390880672199767, Test 3.335310459136963
Epoch 213, lr 3.5e-06
Epoch 213: Train 2.38882371457355, Test 3.3308401107788086
Epoch 214, lr 3.5e-06
Epoch 214: Train 2.3867931678518164, Test 3.32794189453125
Epoch 215, lr 3.5e-06
Epoch 215: Train 2.3848044346914934, Test 3.3236947059631348
Epoch 216, lr 3

Epoch 304, lr 3.5e-06
Epoch 304: Train 2.25200419463659, Test 3.0889458656311035
Epoch 305, lr 3.5e-06
Epoch 305: Train 2.250921744367351, Test 3.086625099182129
Epoch 306, lr 3.5e-06
Epoch 306: Train 2.2497653789985166, Test 3.0843665599823
Epoch 307, lr 3.5e-06
Epoch 307: Train 2.2486893163211104, Test 3.0829873085021973
Epoch 308, lr 3.5e-06
Epoch 308: Train 2.2475557541658757, Test 3.081592082977295
Epoch 309, lr 3.5e-06
Epoch 309: Train 2.2464671379178567, Test 3.0780858993530273
Epoch 310, lr 3.5e-06
Epoch 310: Train 2.2454110950192288, Test 3.077096939086914
Epoch 311, lr 3.5e-06
Epoch 311: Train 2.2442914164270493, Test 3.075735092163086
Epoch 312, lr 3.5e-06
Epoch 312: Train 2.2431862750857556, Test 3.074197769165039
Epoch 313, lr 3.5e-06
Epoch 313: Train 2.2421855753746587, Test 3.0722506046295166
Epoch 314, lr 3.5e-06
Epoch 314: Train 2.241014421535889, Test 3.070554256439209
Epoch 315, lr 3.5e-06
Epoch 315: Train 2.2399402068380625, Test 3.0684468746185303
Epoch 316, lr 3.5

Epoch 404, lr 3.5e-06
Epoch 404: Train 2.159645363354714, Test 2.9499475955963135
Epoch 405, lr 3.5e-06
Epoch 405: Train 2.1589060267126765, Test 2.949181079864502
Epoch 406, lr 3.5e-06
Epoch 406: Train 2.158386307585695, Test 2.9467501640319824
Epoch 407, lr 3.5e-06
Epoch 407: Train 2.1574811825482114, Test 2.9449169635772705
Epoch 408, lr 3.5e-06
Epoch 408: Train 2.1566491028065737, Test 2.945539951324463
Epoch 409, lr 3.5e-06
Epoch 409: Train 2.1559282787860776, Test 2.9436070919036865
Epoch 410, lr 3.5e-06
Epoch 410: Train 2.155095203277779, Test 2.9426708221435547
Epoch 411, lr 3.5e-06
Epoch 411: Train 2.154291924830325, Test 2.941211223602295
Epoch 412, lr 3.5e-06
Epoch 412: Train 2.153590860102959, Test 2.939006805419922
Epoch 413, lr 3.5e-06
Epoch 413: Train 2.1528010790841225, Test 2.939852237701416
Epoch 414, lr 3.5e-06
Epoch 414: Train 2.1520978081995947, Test 2.93841814994812
Epoch 415, lr 3.5e-06
Epoch 415: Train 2.151267753994983, Test 2.9366295337677
Epoch 416, lr 3.5e-0

In [24]:

#all past 29
cur_train = []
for m in models:
    if int(m[8]) == 3: #and int(m[10:]) > 29:
        cur_train.append(m)
test_set = []
train_set = []
pred_set = []

for m in cur_train:
    cur_model = torch.load(os.path.join(path,m)).to('cpu')
    cur_data = get_data(data,m).to('cpu')
    if len(cur_data)< 10000:
        train_set.append(cur_data[0:int(0.8*len(cur_data))])
        test_data = cur_data[-int(0.19*len(cur_data)):]
        test_set.append(test_data)
    else:
        train_set.append(cur_data[0:int(0.8*10000)])

        test_data = cur_data[-int(0.19*10000):]
        test_set.append(test_data)
train_dt = torch.vstack(train_set).to(device)
test_dt = torch.vstack(test_set).to(device)

train_dt = train_dt[torch.count_nonzero(train_dt[:,0:48], dim=1) > 7]
test_dt = test_dt[torch.count_nonzero(test_dt[:,0:48], dim=1) > 7]
size_1_train = np.min([100000,len(train_dt)])
size_1_test = np.min([10000,len(test_dt)])
train_loc = train_dt
test_loc = test_dt
# train = torch.vstack([train,largest[0:1000]] )
train_loc=train_loc[torch.randperm(train_loc.size()[0])]
test_loc=test_loc[torch.randperm(test_loc.size()[0])]
train_loc_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(train_loc)),
    batch_size=100,
    shuffle=False
)

test_loc_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(test_loc)),
    batch_size=100,
    shuffle=False
)

train_1 = train_dt[:,0:48]
test_1 = test_dt[:,0:48]
# train = torch.vstack([train,largest[0:1000]] )
train_1=train_1[torch.randperm(train_1.size()[0])]
test_1=test_1[torch.randperm(test_1.size()[0])]
train_1_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(train_1)),
    batch_size=100,
    shuffle=False
)

test_1_d1_flat = DataLoader(
    TensorDataset(torch.Tensor(test_1)),
    batch_size=100,
    shuffle=False
)

#fine-tune autoencoder
#batch 500

# model_1 = torch.load('models/dt_3_greater_0_450_250_100_dif_2').to(device)
# model_1.decoders.add_module('ReLU',nn.ReLU())
                        
# model_1 = Naive_DAE([48,250,150,16]).to(device)

# model_1 = torch.load('models/dt_1_high_occupancy_greater_7').to(device)

test = test_dt
lr = 3.5e-6
num_epochs = 500

optimizer = optim.Adam(model_1.parameters(), lr,weight_decay=5e-2)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = 0.5)

all_1_test_losses = []
all_1_train_losses = []
# train
running_loss = float("inf")
for epoch in range(num_epochs):
    
    losses = []
    
    for i, data_list in enumerate(train_1_d1_flat):
        model_1.train()
        d = data_list[0]
        v_pred = model_1(d)
        
#         batch_loss = torch.mean(telescopeMSE2(data[:,0:48].to(device), v_pred.to(device)))
        batch_loss = new_loss(d[:,0:48].to(device), v_pred.to(device),epoch,mean, std, cut = 1)
        all_1_train_losses.append(batch_loss.item())
        losses.append(batch_loss.item())
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        
    scheduler.step(batch_loss)
    data_1_test = test[:,0:48]
    model_1.eval()
    test_pred = model_1(data_1_test)
    batch_1_test = new_loss(data_1_test, test_pred,epoch, mean,std, cut =1)
#     batch_1_test = torch.mean(telescopeMSE2(data_1_test[:,0:48].to(device), test_pred.to(device)))
    all_1_test_losses.append(batch_1_test.item())
    running_loss = np.mean(losses)
    running_1est_loss = batch_1_test.item()
    
    print('Epoch {}, lr {}'.format(
        epoch, optimizer.param_groups[0]['lr']))
    
    print(f"Epoch {epoch}: Train {running_loss}, Test {running_1est_loss}")
model_1.eval()
torch.save(model_1,'models/dt_3_high_occupancy_greater_7')

# AE_Stats.gen_all_stats(model_1(dt_1[-size_1_test:,0:48]).cpu(),dt_1[-size_1_test:,0:48].cpu(),dt_1[-size_1_test:].cpu())

Epoch 0, lr 3.5e-06
Epoch 0: Train 23.26238604926278, Test 21.03284454345703
Epoch 1, lr 3.5e-06
Epoch 1: Train 23.078560196911848, Test 20.94330596923828
Epoch 2, lr 3.5e-06
Epoch 2: Train 24.769778074256678, Test 50.632225036621094
Epoch 3, lr 3.5e-06
Epoch 3: Train 24.662861933139126, Test 50.049434661865234
Epoch 4, lr 3.5e-06
Epoch 4: Train 24.56350128581985, Test 49.744346618652344
Epoch 5, lr 3.5e-06
Epoch 5: Train 24.47180345794301, Test 49.51152038574219
Epoch 6, lr 3.5e-06
Epoch 6: Train 24.383423649038306, Test 49.269927978515625
Epoch 7, lr 3.5e-06
Epoch 7: Train 24.29495472849151, Test 49.09344482421875
Epoch 8, lr 3.5e-06
Epoch 8: Train 24.211919100883076, Test 48.96323013305664
Epoch 9, lr 3.5e-06
Epoch 9: Train 23.92461583251325, Test 48.136234283447266
Epoch 10, lr 3.5e-06
Epoch 10: Train 23.778733326476296, Test 47.99531555175781
Epoch 11, lr 3.5e-06
Epoch 11: Train 23.68412759333481, Test 47.930419921875
Epoch 12, lr 1.75e-06
Epoch 12: Train 23.598924468868553, Test 

Epoch 101, lr 2.1875e-07
Epoch 101: Train 22.25706271301081, Test 45.90440368652344
Epoch 102, lr 2.1875e-07
Epoch 102: Train 22.251922719272567, Test 45.89717102050781
Epoch 103, lr 2.1875e-07
Epoch 103: Train 22.24715802541976, Test 45.891029357910156
Epoch 104, lr 2.1875e-07
Epoch 104: Train 22.242304887889343, Test 45.88499450683594
Epoch 105, lr 2.1875e-07
Epoch 105: Train 22.237258436650407, Test 45.877906799316406
Epoch 106, lr 2.1875e-07
Epoch 106: Train 22.232435296690515, Test 45.86994171142578
Epoch 107, lr 2.1875e-07
Epoch 107: Train 22.227774977193448, Test 45.86591720581055
Epoch 108, lr 2.1875e-07
Epoch 108: Train 22.22261652181178, Test 45.856231689453125
Epoch 109, lr 2.1875e-07
Epoch 109: Train 22.21819334736577, Test 45.854286193847656
Epoch 110, lr 2.1875e-07
Epoch 110: Train 22.212978681713466, Test 45.84544372558594
Epoch 111, lr 2.1875e-07
Epoch 111: Train 22.208313096992274, Test 45.83763122558594
Epoch 112, lr 2.1875e-07
Epoch 112: Train 22.20341897619114, Test

Epoch 198, lr 2.1875e-07
Epoch 198: Train 21.80585293397001, Test 45.29244613647461
Epoch 199, lr 2.1875e-07
Epoch 199: Train 21.801103062492338, Test 45.28429412841797
Epoch 200, lr 2.1875e-07
Epoch 200: Train 21.796596824583204, Test 45.27732849121094
Epoch 201, lr 2.1875e-07
Epoch 201: Train 21.792461804597956, Test 45.273414611816406
Epoch 202, lr 2.1875e-07
Epoch 202: Train 21.787665405979862, Test 45.26509094238281
Epoch 203, lr 2.1875e-07
Epoch 203: Train 21.783371362960878, Test 45.26097106933594
Epoch 204, lr 2.1875e-07
Epoch 204: Train 21.77879072589639, Test 45.255409240722656
Epoch 205, lr 2.1875e-07
Epoch 205: Train 21.774426172201526, Test 45.245140075683594
Epoch 206, lr 2.1875e-07
Epoch 206: Train 21.769911841011833, Test 45.24238586425781
Epoch 207, lr 2.1875e-07
Epoch 207: Train 21.765453429673435, Test 45.23357391357422
Epoch 208, lr 2.1875e-07
Epoch 208: Train 21.760988706031455, Test 45.22772216796875
Epoch 209, lr 2.1875e-07
Epoch 209: Train 21.756668426764847, Te

Epoch 295, lr 2.1875e-07
Epoch 295: Train 21.384913696084983, Test 44.67402267456055
Epoch 296, lr 2.1875e-07
Epoch 296: Train 21.380800262105808, Test 44.66992950439453
Epoch 297, lr 2.1875e-07
Epoch 297: Train 21.376691030180503, Test 44.66291046142578
Epoch 298, lr 2.1875e-07
Epoch 298: Train 21.372350878774384, Test 44.65586853027344
Epoch 299, lr 2.1875e-07
Epoch 299: Train 21.36816791919033, Test 44.651519775390625
Epoch 300, lr 2.1875e-07
Epoch 300: Train 21.364169375886643, Test 44.642723083496094
Epoch 301, lr 2.1875e-07
Epoch 301: Train 21.359970413412086, Test 44.63525390625
Epoch 302, lr 2.1875e-07
Epoch 302: Train 21.355807381304203, Test 44.628822326660156
Epoch 303, lr 2.1875e-07
Epoch 303: Train 21.35157749662674, Test 44.627349853515625
Epoch 304, lr 2.1875e-07
Epoch 304: Train 21.3477649382603, Test 44.617523193359375
Epoch 305, lr 2.1875e-07
Epoch 305: Train 21.343443595823437, Test 44.61308670043945
Epoch 306, lr 2.1875e-07
Epoch 306: Train 21.339384439256456, Test 

Epoch 392, lr 2.1875e-07
Epoch 392: Train 20.99133679111308, Test 44.10886764526367
Epoch 393, lr 2.1875e-07
Epoch 393: Train 20.987336500583854, Test 44.10325622558594
Epoch 394, lr 2.1875e-07
Epoch 394: Train 20.98340030167819, Test 44.09770202636719
Epoch 395, lr 2.1875e-07
Epoch 395: Train 20.979719260101945, Test 44.093658447265625
Epoch 396, lr 2.1875e-07
Epoch 396: Train 20.975581774299528, Test 44.08563995361328
Epoch 397, lr 2.1875e-07
Epoch 397: Train 20.971673635396446, Test 44.08051300048828
Epoch 398, lr 2.1875e-07
Epoch 398: Train 20.96781377007441, Test 44.07279968261719
Epoch 399, lr 2.1875e-07
Epoch 399: Train 20.964026085731913, Test 44.07106018066406
Epoch 400, lr 2.1875e-07
Epoch 400: Train 20.959940646607198, Test 44.062408447265625
Epoch 401, lr 2.1875e-07
Epoch 401: Train 20.955988378485536, Test 44.0573616027832
Epoch 402, lr 2.1875e-07
Epoch 402: Train 20.952331411102673, Test 44.053768157958984
Epoch 403, lr 2.1875e-07
Epoch 403: Train 20.94835222110827, Test 

Epoch 489, lr 2.1875e-07
Epoch 489: Train 20.617162966630097, Test 43.564453125
Epoch 490, lr 2.1875e-07
Epoch 490: Train 20.613393258459773, Test 43.5606803894043
Epoch 491, lr 2.1875e-07
Epoch 491: Train 20.609833967342297, Test 43.55609130859375
Epoch 492, lr 2.1875e-07
Epoch 492: Train 20.605966381967804, Test 43.55286407470703
Epoch 493, lr 2.1875e-07
Epoch 493: Train 20.602225025789238, Test 43.54607391357422
Epoch 494, lr 2.1875e-07
Epoch 494: Train 20.59858850667506, Test 43.542236328125
Epoch 495, lr 2.1875e-07
Epoch 495: Train 20.59498234344608, Test 43.53724670410156
Epoch 496, lr 2.1875e-07
Epoch 496: Train 20.591072412184726, Test 43.53093719482422
Epoch 497, lr 2.1875e-07
Epoch 497: Train 20.587538749004096, Test 43.526954650878906
Epoch 498, lr 2.1875e-07
Epoch 498: Train 20.58378793653637, Test 43.523826599121094
Epoch 499, lr 2.1875e-07
Epoch 499: Train 20.58012461524932, Test 43.51557159423828


# Question


In [1]:
AE_Stats.gen_all_stats(test_pred.cpu(),data_1_test.cpu(),data_1_test.cpu())


NameError: name 'AE_Stats' is not defined